In [1]:
!pip install pydicom 
!pip install pylibjpeg[all]

     |████████████████████████████████| 2.0 MB 11.9 MB/s 
     |████████████████████████████████| 4.3 MB 18.6 MB/s 
     |████████████████████████████████| 1.3 MB 58.4 MB/s 
     |████████████████████████████████| 969 kB 28.9 MB/s 


In [6]:
!git clone https://github.com/omerenesbayram/lymph-node-metastasis.git

Cloning into 'lymph-node-metastasis'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 34 (delta 4), reused 26 (delta 3), pack-reused 0
Unpacking objects: 100% (34/34), done.
fatal: not a git repository (or any of the parent directories): .git


In [10]:
%cd lymph-node-metastasis/
!git checkout initialization

/content/lymph-node-metastasis
Branch 'initialization' set up to track remote branch 'initialization' from 'origin'.
Switched to a new branch 'initialization'


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import numpy as np

from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.optimizers import Adam

import config
from model import resnet
from data_generator.data_generator import DataGenerator
from data_generator.helper import excel

In [12]:
pos, neg = excel.get_names(config.PATIENT_LIST_PATH)

In [13]:
neg = neg[:len(pos)]

In [14]:
train_names_pos, val_names_pos = tts(pos, test_size=0.2, random_state=42)
train_names_neg, val_names_neg = tts(neg, test_size=0.2, random_state=42)

In [15]:
train_names = np.asarray(train_names_pos + train_names_neg)
np.random.shuffle(train_names)
val_names = np.asarray(val_names_pos + val_names_neg)
np.random.shuffle(val_names)

In [16]:
print(f'Number of Patients for Training: {len(train_names)}')
print(f'Number of Patients for Validation: {len(val_names)}')

Number of Patients for Training: 48
Number of Patients for Validation: 12


In [17]:
train_gen = DataGenerator(train_names, "train", config.PATIENT_LIST_PATH, batch_size=4)
val_gen = DataGenerator(val_names, "train", config.PATIENT_LIST_PATH, batch_size=4)

In [18]:
model = resnet.build_resnet_18(input_shape=(100, 100, 100, 1), num_outputs=2, reg_factor=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])

In [19]:
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir=config.LOG_PATH)

In [20]:
model.fit(train_gen, batch_size=4, epochs=10, validation_data=val_gen, callbacks = [tensorboard], verbose=2)

Epoch 1/10


KeyboardInterrupt: ignored

In [ ]:
model.save(config.MODEL_PATH)